NOMBRES: Mariel Alejandra 

APELLIDOS: Guamuche Recinos

CARNE: 21150

FECHA: 06/08/2025

**Instrucciones:**

El objetivo de esta práctica es que apliquen lo visto en clase. Al texto "metamorphosis_kafka" que pueden encontrarlo en el siguiente link: https://www.gutenberg.org/ebooks/5200 (Usar Plain Text)
deben aplicarse técnicas de tokenización, lemmatización, stemming y normalización. Sin embargo, estos procesos no se aplican de manera indiscriminada. Para este proyecto tienen que aplicar las técnicas a **1 de 2 objetivos**.

1. Este texto se usará para entrenar un LLM para que escriba con un estilo kafkiano.
2. Reconocer textos escritor por Kafka. El objetivo final en esta línea sería poder etiquetar algo como "kafka" y "no kafka".

Para ambas perspectivas qué tipos de tokenización, lemmatización, stemming y normalización utilizaría. Ej: tokenizaría por oraciones en lugares de palabras, tomaría o no en cuenta signos de puntuación, removería stopwords o no, etc. Si para esto consulta otras fuentes, citelas.

Luego de explicar ambas perspectivas **escoja 1** y programe cada una de las etapas.

Se recomienda que antes de empezar, revise el texto para ver si hay cosas que le conviene editar antes de empezar a programar.

### Características del texto:
- Se evidencian oraciones extensas, con varios signos de puntuación.
- Los diálogos entre personajes están señalados entre comillas dobles.
- En la traducción del texto proporcionado, se utilizan gran variedad de contracciones en inglés "don't"->"do not".
- De acuerdo a muchos que describen el estilo kafkiano, se trata de textos que abarcan la temática existencial, hasta cierto punto fatalistas. Además de la represión del individuo y en análisis instrospectivo, que puede confundir al lector. 

*El texto proporcionado incluye al principio y al final información del libro, datos que fueron eliminados manualmente*
## Pipeline para el segundo modelo: clasificación de textos
### Descripción del pipeline a implementar
1. Normalización
Se utiliza normalización unicode y lowercasing con el fin de unificar diferentes codificaciones y evitar crear tokens con mayúsuculas y minúsculas. 
Adicionalmente, se ha implementado la expansión de contracciones para evitar que el modelo trate a "don't" como token distinto de "do not". Esto se espera mejora la claridad semántica.

2. Eliminación de puntuación y caracteres no alfabéticos
Usando expresiones regulares se elimina la puntuación que no aporta significado semántico al modelo de clasificación y genera tokens irrelevantes. A pesar que las estructuras de uso "exagerado" de puntuación en los textos de Kafka, esto podría confundir al modelo donde expresiones como "¡¡¡Feliz cumpleaños!!! eres la mejor..." podría clasificarlos como de Kafka.

3. Tokenización
Tokenización por palabras para operar sobre cada token los demás pasos del pipeline. Se considero hacer separación por oraciones para tratar de tener features por oraciones en la escritura de Kafka, pero esto implicaría un alto consumo de memoria y el modelo no aprendería (posiblemente) las generalidades de la redacción de Kafka. Donde además se reduciría el vocabulario.

4. Eliminación de stopwords

5. Stemming 
Usando PorterStemmer se reduce variantes para compactar el vocabulario. Se utiliza Porter Stemmer porque el texto está en inglés y permite agrupar palabras similares.
Algunas páginas recomiendan utilizar más stemming que lemmatización si se trata de un modelo de clasificación.
https://kazumatsuda.medium.com/nlp-preprocessing-text-data-part-1-b4641af2a5af
https://mylearningsinaiml.wordpress.com/nlp/data-preparation/clean-text/ 

6. Lematización 
Usando el etiqueta POS básico, se aplica lematización para mapear cada término a su forma canónica y es opcional para el código.

Para la implementación, se ha puesto un criterio de si se desea manejar con lematización o stemming o ambas.

7. Reconstrucción
Opción para unir los tokens en un string o dejarlos coom lista de palabras.


#### Implementación

In [ ]:
# Librería para expandir las contracciones
# !pip install contractions

In [36]:
import re
import unicodedata
import nltk
import contractions

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ, ADV

# Mapeo de etiquetas POS de nltk.pos_tag a wordnet
POS_MAP = {
    'J': ADJ,
    'V': VERB,
    'N': NOUN,
    'R': ADV
}

[nltk_data] Downloading package punkt to C:\Users\Mariel
[nltk_data]     Guamuche\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mariel
[nltk_data]     Guamuche\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Mariel
[nltk_data]     Guamuche\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mariel Guamuche\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Mariel
[nltk_data]     Guamuche\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [37]:
def normalize_text(text: str) -> str:
    nkfd = unicodedata.normalize('NFKD', text)
    no_accents = ''.join(c for c in nkfd if not unicodedata.combining(c))
    return no_accents.lower()

def expand_contractions(text: str) -> str:
    # usa la librería contractions para convertir contracciones en su forma completa
    return contractions.fix(text)

def remove_punctuation(text: str) -> str:
    return re.sub(r'[^a-z\s]', ' ', text)

def tokenize(text: str) -> list[str]:
    return word_tokenize(text)

def remove_stopwords(tokens: list[str]) -> list[str]:
    stops = set(stopwords.words('english'))
    return [t for t in tokens if t not in stops and t.isalpha()]

def stem_tokens(tokens: list[str]) -> list[str]:
    stemmer = PorterStemmer()
    return [stemmer.stem(t) for t in tokens]

def lemmatize_tokens(tokens: list[str]) -> list[str]:
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(tokens)
    lemmata = []
    for token, tag in pos_tags:
        pos = POS_MAP.get(tag[0], NOUN)
        lemmata.append(lemmatizer.lemmatize(token, pos=pos))
    return lemmata

def preprocess_pipeline(text: str,
                        do_stem: bool = True,
                        do_lemmatize: bool = True) -> list[str]:
    """
    Pipeline completo:
    1. Normalizar y pasar a minúsculas
    1.5 Expandir contracciones
    2. Quitar puntuación
    3. Tokenizar
    4. Eliminar stopwords
    5. Stemming (opcional)
    6. Lematización (opcional)
    """
    # 1. Normalizar
    t = normalize_text(text)
    # 1.5 Expandir contracciones
    t = expand_contractions(t)
    # 2. Quitar puntuación
    t = remove_punctuation(t)
    # 3. Tokenizar
    tokens = tokenize(t)
    # 4. Eliminar stopwords y no-alfabéticos
    tokens = remove_stopwords(tokens)
    # 5. Stemming
    if do_stem:
        tokens = stem_tokens(tokens)
    # 6. Lematización
    if do_lemmatize:
        tokens = lemmatize_tokens(tokens)
    return tokens

In [38]:
import numpy as np
# Ejemplo de uso:
with open('pg5200.txt', 'r', encoding='utf-8') as f:
    sample = f.read()
tokens = preprocess_pipeline(sample)
import collections
cnt = collections.Counter(tokens)
# Calculate stylistic features
features = {
    'num_tokens': len(tokens),
    'num_unique_tokens': len(cnt),
    'vocab_size': len(cnt),
    'most_common': cnt.most_common(20),
    'avg_tokens_per_text': float(np.mean([len(doc) for doc in [tokens]]))
}

In [39]:
features

{'num_tokens': 10035,
 'num_unique_tokens': 1838,
 'vocab_size': 1838,
 'most_common': [('gregor', 298),
  ('would', 204),
  ('room', 133),
  ('could', 127),
  ('even', 102),
  ('father', 102),
  ('sister', 101),
  ('door', 97),
  ('mother', 90),
  ('go', 86),
  ('back', 83),
  ('make', 76),
  ('time', 74),
  ('one', 73),
  ('say', 73),
  ('get', 71),
  ('come', 69),
  ('way', 63),
  ('look', 61),
  ('take', 55)],
 'avg_tokens_per_text': 10035.0}